In [1]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, NamedStyle
from openpyxl.utils import get_column_letter
from datetime import datetime
# import win32com.client as win32
import tkinter as tk
from tkinter import ttk, messagebox
# import math

# Define the report date
report_date = datetime(
    2026, # Year
    2,   # Month
    25    # Day
)

# DMS Sellout Setting
# Y - if you want to monitor RD Initiated Product Codes only.
# N - if you want to monitor all products.
dummy_code = "Y"

# Generate filenames based on the report date
def generate_filename(report_type: str, date_obj: datetime, prefix: str = "KENEA") -> str:
    """Generate Excel filename with format: 'KENEA Report_Type_Month D, YYYY.xlsx'"""
    # Use '#' instead of '-' for Windows compatibility
    formatted_date = date_obj.strftime("%B %d, %Y").replace(" 0", " ")  # remove leading zero
    return f"{prefix} {report_type}_{formatted_date}.xlsx"

# Test the function
Net_Invoiced = generate_filename("Net Invoiced", report_date)
ex_filename_so = generate_filename("Sales Order", report_date)
ex_filename_ser_inv = generate_filename("Served Invoice", report_date)


def generate_export_path(base_dir: str, date_obj: datetime, subfolder: str) -> str:
    year = date_obj.strftime("%Y")
    month_name = date_obj.strftime("%m.%B")
    month_check = date_obj.strftime("%m").lstrip('0')  # remove leading zeros
    day_check = date_obj.strftime("%d").lstrip('0')    # remove leading zeros
    year_check = date_obj.strftime("%Y")
    day_folder = f"{month_check}.{day_check}.{year_check[2:]}"

    return os.path.join(base_dir, year, month_name, day_folder, subfolder)

base_dir = r"C:\Users\User\Downloads\COB DATA"

# Define paths and filenames
export_path = generate_export_path(base_dir, report_date, "Exports")

# Define import path
import_path = generate_export_path(base_dir, report_date, "COB")

# Ensure export directory exists
os.makedirs(export_path, exist_ok=True)

# Load reference data
category_ref_path = r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\CATEGORY.xlsx"
category_df = pd.read_excel(category_ref_path)

field_supervisors_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\Field Supervisors.xlsx")
week_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\WEEK.xlsx")

# Define import filenames
im_filename_inv = "DMS-Invoice-on.xlsx"
im_filename_ret = "DMS-Customer Returns-on.xlsx"
im_filename_cust = "DMS-Customer-on.xlsx"
im_filename_pl = "DMS-Price-on.xlsx"
im_filename_so = "DMS-Sales Order-on.xlsx"

# Define full import paths
Net_Invoiced_path = os.path.join(export_path, Net_Invoiced)


def generate_ci_path(base_dir: str, date_obj: datetime) -> str:
    year = date_obj.strftime("%Y")
    month_name = date_obj.strftime("%m.%B")

    return os.path.join(base_dir, year, month_name)

ci_path = generate_ci_path(base_dir, report_date)

In [2]:
#Reading MONTHLY WRONG C.I. MONITORING

ci_import_path = os.path.join(ci_path, "MONTHLY WRONG C.I. MONITORING.xlsx")
ci_df = pd.read_excel(ci_import_path, skiprows=4)

ci_row_count = ci_df.count()

print(f"Numbers of Recorded Wrong C.I.: {ci_row_count.iloc[0]}")

ci_checker_inv = ci_df.copy()
ci_checker_inv = ci_checker_inv[['Invoice Filter']]

ci_checker_ret = ci_df.copy()
ci_checker_ret = ci_checker_ret[['Customer Return Filter']]



Numbers of Recorded Wrong C.I.: 10


In [3]:
#INVOICE ON Arrangement

full_im_path_inv = os.path.join(import_path, im_filename_inv)

df1 = pd.read_excel(full_im_path_inv)

dfc1 = df1[['Invoice Date', 'Sold To Customer Number',
'Product Code', 'Product/Item Description',
'Total Item amount with Tax and Discount', 'Invoice Item Type','Invoice number']]
inv = dfc1[dfc1['Invoice Item Type'] != 'ITM_SALES_TAX']

if not ci_checker_inv.empty:
    ci_inv = inv.copy()
    ci_inv.insert(7,'Invoice Filter',0)
    ci_inv['Invoice Filter'] = ci_inv['Invoice number'].astype(str) + "_" + ci_inv['Product Code'].astype(str)
    ci_inv_processed = ci_inv[~ci_inv['Invoice Filter'].isin(ci_checker_inv['Invoice Filter'])]
    inv = ci_inv_processed.drop('Invoice Filter', axis=1)

inv_f = inv.copy()
inv_f = inv_f.drop('Invoice number', axis=1)
inv_f.insert(6,'BO',0)
inv_f.insert(7,'FG',0)
inv_f['Total Item amount with Tax and Discount'] = inv_f['Total Item amount with Tax and Discount'].fillna(0)
inv_f['Product Code'] = inv_f['Product Code'].astype(str)
inv_f = inv_f.drop('Invoice Item Type', axis=1)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
#CUSTOMER RETURNS Arrangement

full_im_path_ret = os.path.join(import_path, im_filename_ret)

df2 = pd.read_excel(full_im_path_ret)

dfc2 = df2[['Customer Return Date', 'Sold To Customer Number', 'Product Code',
            'Product Description', 'Facility Name', 'Estimated Product Return Amount','Customer Return Number']].copy()
dfc2['Estimated Product Return Amount'] = dfc2['Estimated Product Return Amount'].fillna(0)
ret_init = dfc2

if not ci_checker_ret.empty:
    ci_ret = ret_init.copy()
    ci_ret.insert(7,'Customer Return Filter',0)
    ci_ret['Customer Return Filter'] = ci_ret['Customer Return Number'].astype(str) + "_" + ci_ret['Product Code'].astype(str)
    ci_ret_processed = ci_ret[~ci_ret['Customer Return Filter'].isin(ci_checker_ret['Customer Return Filter'])]
    ret_init = ci_ret_processed.drop('Customer Return Filter', axis=1)

dfcr = ret_init.copy()
dfcr = dfcr.drop('Customer Return Number', axis=1)
dfcr.insert(4,'Total Item amount with Tax and Discount', 0)
dfcr.insert(7,'with vat', 0)
dfcr['with vat'] = dfcr['Estimated Product Return Amount'] * 1.12
dfc3 = dfcr.copy()

if dfc3['Facility Name'].str.contains('Virtual').any():
    dfc3['Facility Name'] = dfc3['Facility Name'].str.replace('Virtual', 'FG')

# Extract only 'FG' or 'BO'
dfc3['Facility Name'] = dfc3['Facility Name'].str.extract(r'\b(FG|BO)\b')

cust_ret = dfc3.pivot_table(index=['Customer Return Date', 'Sold To Customer Number', 
'Product Code', 'Product Description', 'Total Item amount with Tax and Discount'], columns='Facility Name', values='with vat', aggfunc=sum)

ret_f = cust_ret.reset_index()

ret_f['Product Code'] = ret_f['Product Code'].astype(str)
if 'BO' in ret_f.columns and 'FG' not in ret_f.columns:
    ret_f['BO'] = ret_f['BO'].fillna(0)
    ret_f['FG'] = 0.0
elif 'FG' in ret_f.columns and 'BO' not in ret_f.columns:
    ret_f['FG'] = ret_f['FG'].fillna(0)
    ret_f['BO'] = 0.0
else:
    ret_f['BO'] = ret_f['BO'].fillna(0)
    ret_f['FG'] = ret_f['FG'].fillna(0)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_15444\2808883817.py:32: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cust_ret = dfc3.pivot_table(index=['Customer Return Date', 'Sold To Customer Number',


In [5]:
# SALES ORDERS Arrangements

full_im_path_so = os.path.join(import_path, im_filename_so)

df3 = pd.read_excel(full_im_path_so)

dfc3 = df3[['Last Modified Date', 'Sold To Customer number',
 'Product Code', 'Product Description', 'Total Product Amount', 'SO status', 'SO Number']]
so_par = dfc3[dfc3['SO status'] == 'Invoiced']

#Get Original Invoice
dfc = df1[['Invoice Item Type','Product Code', 'SO Number','Invoice number']]
inv_so = dfc[dfc['Invoice Item Type'] != 'ITM_SALES_TAX']

if not ci_checker_inv.empty:
    inv_so = inv_so.copy()
    inv_so.insert(4,'Invoice Filter',0)
    inv_so.insert(5,'SO Filter',0)

    inv_so['Invoice Filter'] = inv_so['Invoice number'].astype(str) + "_" + inv_so['Product Code'].astype(str)
    inv_so['SO Filter'] = inv_so['SO Number'].astype(str) + "_" + inv_so['Product Code'].astype(str)

    inv_so_processed = inv_so[inv_so['Invoice Filter'].isin(ci_checker_inv['Invoice Filter'])]
    
    ci_so = inv_so_processed[['SO Filter']]

    inv_so = so_par.copy()
    inv_so.insert(7,'SO Filter',0)
    inv_so['SO Filter'] = inv_so['SO Number'].astype(str) + "_" + inv_so['Product Code'].astype(str)
    ci_so_processed = inv_so[~inv_so['SO Filter'].isin(ci_so['SO Filter'])]
    so_par = ci_so_processed.drop('SO Filter', axis=1)

so_par = so_par.drop('SO Number', axis=1)
so_par['Total Product Amount'] = so_par['Total Product Amount'].fillna(0)
so_par.insert(6,'with vat', 0)
so_par['with vat'] = so_par['Total Product Amount'] * 1.12

so_f = so_par.copy()
so_f = so_f.drop(['Total Product Amount', 'SO status'], axis=1)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
#Customer Master List

full_im_path_cust = os.path.join(import_path, im_filename_cust)

cust_df = pd.read_excel(full_im_path_cust)

cust_df = cust_df[[
    'NEXT_UP_NUMBER',
    'CUSTOMER_NAME',
    'PARTY_CLASSIFICATION_DESCRIPTION',
    'KEY_ACCOUNT',
    'SALES_REP_ID',
    'SALES_REP_NAME',
    'BRANCH_NAME',
    'GEO_LOCATION_HIERARCHYDESCRIPTION',
    'CITY',
    'STATE_PROVINCE',
    'CHANNEL'
]]

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
#Price List Arrangements

full_im_path_pl = os.path.join(import_path, im_filename_pl)

pl_df = pd.read_excel(full_im_path_pl)

pl_df2 = pl_df

pl_df2 = pl_df2[['product_code', 'product_description', 'uom_description', 'selling_price', 'cust_class', 'cust_channel']]
pl_df2.insert(4,'with vat', 0)
pl_df2['with vat'] = pl_df2['selling_price'] * 1.12
pl_df3 = pl_df2[pl_df2['cust_class'] != 'BEV Dealer']

pl_df_m0 = pl_df3
pl_df_m2 = pl_df3

pl_df_m0 = pl_df_m0[pl_df_m0['cust_channel'] != 'VAN(EXTRUCK)']
pl_df_m2 = pl_df_m2[pl_df_m2['cust_channel'] == 'VAN(EXTRUCK)']

#Price List M0

pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description',], columns='uom_description', values='with vat', aggfunc=sum)

pl_df_m0_f_1 = pl_df_m0_f.reset_index()

pl_df_m0_f_1['Case'] = pl_df_m0_f_1['Case'].fillna(0)
pl_df_m0_f_1['Piece'] = pl_df_m0_f_1['Piece'].fillna(0)
pl_df_m0_f_1['Subcase'] = pl_df_m0_f_1['Subcase'].fillna(0)

pl_m0_final = pl_df_m0_f_1[['product_code', 'product_description', 'Case', 'Subcase', 'Piece']]
pl_m0_final['product_code'] = pl_m0_final['product_code'].astype(str)

ex_filename_pl_m0 = "M0 Pricelist.xlsx"
full_path_pl_m0 = os.path.join(export_path, ex_filename_pl_m0)

export_pl_m0 = pl_m0_final
export_pl_m0.to_excel(full_path_pl_m0, index=False)

# Set author metadata
wb_m0 = load_workbook(full_path_pl_m0)
wb_m0.properties.author = "Joshua Ocampo"
wb_m0.save(full_path_pl_m0)

#Price List M2

pl_df_m2_f = pl_df_m2.pivot_table(index=['product_code', 'product_description',], columns='uom_description', values='with vat', aggfunc=sum)

pl_df_m2_f_1 = pl_df_m2_f.reset_index()

pl_df_m2_f_1['Case'] = pl_df_m2_f_1['Case'].fillna(0)
pl_df_m2_f_1['Piece'] = pl_df_m2_f_1['Piece'].fillna(0)
pl_df_m2_f_1['Subcase'] = pl_df_m2_f_1['Subcase'].fillna(0)

pl_m2_final = pl_df_m2_f_1[['product_code', 'product_description', 'Case', 'Subcase', 'Piece']]
pl_m2_final['product_code'] = pl_m2_final['product_code'].astype(str)

ex_filename_pl_m2 = "M2 Pricelist.xlsx"
full_path_pl_m2 = os.path.join(export_path, ex_filename_pl_m2)

export_pl_m2 = pl_m2_final
export_pl_m2.to_excel(full_path_pl_m2, index=False)

# Set author metadata
wb_m2 = load_workbook(full_path_pl_m2)
wb_m2.properties.author = "Joshua Ocampo"
wb_m2.save(full_path_pl_m2)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_15444\3862981563.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_df2['with vat'] = pl_df2['selling_price'] * 1.12
C:\Users\User\AppData\Local\Temp\ipykernel_15444\3862981563.py:22: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description

In [8]:
# Function to edit missing categories via GUI

# Prepare data for category checking
cat_ck = category_df.copy()
inv_ck = inv_f[['Product Code', 'Product/Item Description']].rename(
    columns={'Product Code': 'SKU CODE', 'Product/Item Description': 'SKU NAME'}
).drop_duplicates(subset=['SKU CODE']).reset_index(drop=True)

ret_ck = ret_f[['Product Code', 'Product Description']].rename(
    columns={'Product Code': 'SKU CODE', 'Product Description': 'SKU NAME'}
).drop_duplicates(subset=['SKU CODE']).reset_index(drop=True)

so_ck = so_f[['Product Code', 'Product Description']].rename(
    columns={'Product Code': 'SKU CODE', 'Product Description': 'SKU NAME'}
).drop_duplicates(subset=['SKU CODE']).reset_index(drop=True)

# do NOT pre-merge the category reference into all_products here — keep SKU NAME coming from source tables
all_products = pd.concat([inv_ck, ret_ck, so_ck], ignore_index=True)
all_products = all_products.drop_duplicates(subset=['SKU CODE']).reset_index(drop=True)

def edit_missing_categories(category_df, products_df, category_ref_path):
    """
    GUI to fill blank CATEGORY for SKUs.
    Ensures SKU NAME is shown in the popup and saved into the reference file.
    Returns (updated_category_df, merged_products_df)
    """
    products = products_df.copy()
    # ensure SKU CODE/NAME exist and are strings
    products['SKU CODE'] = products['SKU CODE'].astype(str)
    if 'SKU NAME' not in products.columns:
        products['SKU NAME'] = ''
    products['SKU NAME'] = products['SKU NAME'].fillna('').astype(str)

    # work on a clean copy of the category reference
    cat = category_df.copy()
    if 'SKU CODE' not in cat.columns:
        cat['SKU CODE'] = ''
    cat['SKU CODE'] = cat['SKU CODE'].fillna('').astype(str)
    if 'CATEGORY' not in cat.columns:
        cat['CATEGORY'] = None
    if 'SKU NAME' not in cat.columns:
        cat['SKU NAME'] = ''
    cat['SKU NAME'] = cat['SKU NAME'].fillna('').astype(str)

    # build merged view that maps CATEGORY from reference into products (do not pull SKU NAME from cat for display)
    merged = products.copy()
    cat_map = cat.set_index('SKU CODE')['CATEGORY']
    merged['CATEGORY'] = merged['SKU CODE'].map(cat_map)

    # find SKUs with missing category
    missing_mask = merged['CATEGORY'].isna() | merged['CATEGORY'].astype(str).str.strip().eq('')
    missing = merged.loc[missing_mask, ['SKU CODE', 'SKU NAME']].drop_duplicates().reset_index(drop=True)

    # if missing.empty:
    #     try:
    #         root = tk.Tk()
    #         root.withdraw()
    #         messagebox.showinfo("No Missing Categories", "No blank values found in 'CATEGORY'.", parent=root)
    #         root.destroy()
    #     except Exception:
    #         print("No missing categories.")
    #     return cat, merged
    
    if missing.empty:
        print("No Missing Categories: No blank values found in 'CATEGORY'.")
        return cat, merged

    existing_categories = sorted([c for c in cat['CATEGORY'].dropna().astype(str).unique() if str(c).strip() != ''])

    # Build GUI
    root = tk.Tk()

    root.wm_attributes('-topmost', 1)
    root.lift()
    root.focus_force()

    root.title("Fill missing CATEGORY entries")
    root.geometry("495x360")

    container = ttk.Frame(root)
    container.pack(fill=tk.BOTH, expand=True)

    canvas = tk.Canvas(container)
    v_scroll = ttk.Scrollbar(container, orient="vertical", command=canvas.yview)
    inner = ttk.Frame(canvas)

    inner.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    canvas.create_window((0, 0), window=inner, anchor="nw")
    canvas.configure(yscrollcommand=v_scroll.set)

    canvas.grid(row=0, column=0, sticky="nsew")
    v_scroll.grid(row=0, column=1, sticky="ns")
    container.rowconfigure(0, weight=1)
    container.columnconfigure(0, weight=1)

    headers = ("Product Code", "Product Description", "Category (pick or type)")
    for c, h in enumerate(headers):
        lbl = ttk.Label(inner, text=h, font=("Segoe UI", 10, "bold"), anchor="w")
        lbl.grid(row=0, column=c, padx=6, pady=6, sticky="w")

    widgets = []  # (sku_code, sku_name, combobox)

    for i, row in missing.iterrows():
        r = i + 1
        sku = str(row['SKU CODE'])
        name = '' if pd.isna(row.get('SKU NAME', '')) else str(row.get('SKU NAME', ''))
        ttk.Label(inner, text=sku).grid(row=r, column=0, padx=6, pady=4, sticky="w")
        ttk.Label(inner, text=name).grid(row=r, column=1, padx=6, pady=4, sticky="w")
        cb = ttk.Combobox(inner, values=existing_categories, width=20)
        cb.set('')  # start empty
        cb.grid(row=r, column=2, padx=6, pady=4, sticky="w")
        widgets.append((sku, name, cb))

    def on_save():
        nonlocal cat, merged, existing_categories
        updated_any = False

        # make sure SKU NAME exists as string column in cat
        if 'SKU NAME' not in cat.columns:
            cat['SKU NAME'] = ''
        cat['SKU NAME'] = cat['SKU NAME'].fillna('').astype(str)

        cat_cols = list(cat.columns)

        for sku, sku_name, cb in widgets:
            val = cb.get().strip()
            if not val:
                continue
            updated_any = True
            sku_str = str(sku)
            mask = cat['SKU CODE'].astype(str) == sku_str
            if mask.any():
                # update existing rows: always set CATEGORY, update SKU NAME if empty or different and provided
                cat.loc[mask, 'CATEGORY'] = val
                to_update_name = (cat['SKU NAME'].astype(str).str.strip() == '') | (cat['SKU NAME'].astype(str) != sku_name)
                cat.loc[mask & to_update_name, 'SKU NAME'] = sku_name
            else:
                # create new row preserving columns
                new_row = {c: None for c in cat_cols}
                new_row['SKU CODE'] = sku_str
                new_row['SKU NAME'] = sku_name
                new_row['CATEGORY'] = val
                cat = pd.concat([cat, pd.DataFrame([new_row])], ignore_index=True)

            # keep suggestion list up-to-date for the session
            if val not in existing_categories:
                existing_categories.append(val)
                existing_categories.sort()

        if not updated_any:
            messagebox.showinfo("No changes", "No category values entered.", parent=root)
            return

        # Write reference file and ensure SKU CODE/NAME written
        try:
            cat_out = cat.copy()
            cat_out['SKU CODE'] = cat_out['SKU CODE'].astype(str)
            if 'SKU NAME' not in cat_out.columns:
                cat_out['SKU NAME'] = ''
            cat_out['SKU NAME'] = cat_out['SKU NAME'].fillna('').astype(str)

            # write with commonly expected order if present
            cols_to_write = [c for c in ['SKU CODE', 'SKU NAME', 'CATEGORY'] if c in cat_out.columns] + \
                            [c for c in cat_out.columns if c not in ['SKU CODE', 'SKU NAME', 'CATEGORY']]
            cat_out.to_excel(category_ref_path, index=False, columns=cols_to_write)
        except Exception as e:
            messagebox.showwarning("Save failed", f"Could not write CATEGORY reference file:\n{e}", parent=root)
            return

        # Refresh merged view (map CATEGORY into products and include SKU NAME for downstream)
        cat_map = cat.set_index('SKU CODE')['CATEGORY']
        merged = products.copy()
        merged['CATEGORY'] = merged['SKU CODE'].map(cat_map)

        # also ensure SKU NAME included from products (so callers get SKU NAME available)
        merged['SKU NAME'] = merged['SKU NAME'].fillna('').astype(str)

        messagebox.showinfo("Saved", "Categories updated and saved to reference file.", parent=root)
        try:
            root.wm_attributes('-topmost', 0)
            root.quit()
            root.destroy()
        except Exception:
            try:
                root.destroy()
            except Exception:
                pass

    btn_frame = ttk.Frame(root)
    btn_frame.pack(fill=tk.X, padx=8, pady=8)
    save_btn = ttk.Button(btn_frame, text="Save", command=on_save)
    save_btn.pack(side=tk.RIGHT)
    cancel_btn = ttk.Button(btn_frame, text="Cancel", command=lambda: (root.quit(), root.destroy()))
    cancel_btn.pack(side=tk.RIGHT, padx=(0, 6))

    root.mainloop()
    return cat, merged

# ...existing code...
# Call editor and reload saved reference
category_df, category_checking = edit_missing_categories(category_df, all_products, category_ref_path)

category_df = pd.read_excel(category_ref_path, dtype={'SKU CODE': str})

In [9]:
#COMBINE INVOICED AND RETURNS to NET INVOICED

inv_f = inv_f.copy()
inv_f.rename(columns={
'Invoice Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE',
# 'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE',
'Product/Item Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

ret_f = ret_f.copy()
ret_f.rename(columns={
'Customer Return Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE',
'Product Code': 'SKU CODE', 
'Product Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

net_inv = pd.concat([inv_f, ret_f], axis=0, ignore_index=True)

# Customer Details Arrangements
cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'ACCOUNT CODE'
})

net_inv_f = net_inv.merge(cml_df, on='ACCOUNT CODE', how='left')

# Drop SKU NAME from category_df if it exists to avoid confusion in merges
if 'SKU NAME' in category_df.columns:
    category_df = category_df.drop(['SKU NAME'], axis=1)
net_inv_f_l1 = net_inv_f.merge(category_df,  on='SKU CODE', how='left')

net_inv_f_l2 = net_inv_f_l1.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

net_inv_f_l3 = net_inv_f_l2.merge(week_df, on='DATE', how='inner')

# Get Volume - Price Arrangements

pl_m0_final_reference = pl_m0_final[['product_code', 'Case']].copy()

pl_m0_final_reference = pl_m0_final_reference.rename(columns={
    'product_code': 'SKU CODE',
    'Case': 'SKU PRICE REFERENCE'
})

net_inv_f_l3['SKU CODE'] = net_inv_f_l3['SKU CODE'].str.replace('_old', '', regex=False)

net_inv_f_l4 = net_inv_f_l3.merge(pl_m0_final_reference, on='SKU CODE', how='left')

net_inv_f_l4.insert(5,'VALUE', 0)
net_inv_f_l4['VALUE'] = net_inv_f_l4['SERVED INVOICE'] - net_inv_f_l4['BAD RETURNS'] - net_inv_f_l4['GOOD RETURNS']

# ensure numeric and avoid divide-by-zero when computing VOLUME
net_inv_f_l4['SKU PRICE REFERENCE'] = pd.to_numeric(net_inv_f_l4['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
net_inv_f_l4['VOLUME'] = 0
mask = net_inv_f_l4['SKU PRICE REFERENCE'] != 0
net_inv_f_l4.loc[mask, 'VOLUME'] = net_inv_f_l4.loc[mask, 'VALUE'] / net_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']

# Post-Net Invoiced

export_net_inv = net_inv_f_l4
export_net_inv['RD NAME'] = 'Kimberlin'

export_net_inv = export_net_inv.sort_values(by=['DATE', 'ACCOUNT CODE'])

export_net_inv.rename(columns={
    'RD NAME': 'RD Name',
    'DATE': 'Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'ACCOUNT CODE': 'Sold To Customer number',
    'CUSTOMER_NAME': 'Sold To Customer Name',
    'CATEGORY': 'Category',
    'SKU CODE': 'Product Code',
    'SKU NAME': 'Product Description',
    'VOLUME': 'Volume',
    'VALUE': 'Net Value',
    'GOOD RETURNS': 'Good Stock Returns',
    'BAD RETURNS': 'Bad Stock Returns',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel_Classification',
    'GEO_LOCATION_HIERARCHYDESCRIPTION': 'Brgy',
    'CITY': 'Town',
    'STATE_PROVINCE': 'Province',
    'FS': 'FS',
    'CHANNEL': 'RTM Model'
}, inplace=True)

export_net_inv_final = export_net_inv[[
    'RD Name',
    'Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Net Value',
    'Good Stock Returns',
    'Bad Stock Returns',
    'Channel_Classification',
    'Brgy',
    'Town',
    'Province',
    'FS',
    'RTM Model',
]]

# Net Invoiced Finalization

# Step 1: Export to Excel with a blank row at the top

temp_output = 'temp_file_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    export_net_inv_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 17):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 17):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N', 'O', 'P']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding


# Save changes
ws.sheet_view.showGridLines = False
wb.properties.author = "Joshua Ocampo"
wb.save(Net_Invoiced_path)

C:\Users\User\AppData\Local\Temp\ipykernel_15444\1592696624.py:64: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 0.1235363   0.10295667  0.20589871 ... -0.11       -0.71230111
 -0.64      ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  net_inv_f_l4.loc[mask, 'VOLUME'] = net_inv_f_l4.loc[mask, 'VALUE'] / net_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']


In [10]:
# SALES ORDER Final Arrangements and Export

cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'Sold To Customer number'
})
cat_df = category_df.rename(columns={
    'SKU CODE': 'Product Code'
})
wk_df = week_df.rename(columns={
    'DATE': 'Last Modified Date'
})
pl_df = pl_m0_final_reference.rename(columns={
    'SKU CODE': 'Product Code'
})

sales_orders_df1 = so_f.merge(cml_df, on='Sold To Customer number', how='left')

sales_orders_df2 = sales_orders_df1.merge(cat_df,  on='Product Code', how='left')

sales_orders_df3 = sales_orders_df2.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

sales_orders_df4 = sales_orders_df3.merge(wk_df, on='Last Modified Date', how='inner')

sales_orders_df4['Product Code'] = sales_orders_df4['Product Code'].str.replace('_old', '', regex=False)

sales_orders_df5 = sales_orders_df4.merge(pl_df, on='Product Code', how='left')

sales_orders_df5['SKU PRICE REFERENCE'] = pd.to_numeric(sales_orders_df5['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
sales_orders_df5['VOLUME'] = 0.0
mask = sales_orders_df5['SKU PRICE REFERENCE'] != 0
sales_orders_df5.loc[mask, 'VOLUME'] = sales_orders_df5.loc[mask, 'with vat'] / sales_orders_df5.loc[mask, 'SKU PRICE REFERENCE']

sales_orders_df5['RD NAME'] = 'Kimberlin'

export_sales_order = sales_orders_df5.copy()

export_sales_order.rename(columns={
    'RD NAME': 'RD Name',
    'Last Modified Date': 'SO Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'Sold To Customer number': 'Sold To Customer Number',
    'CUSTOMER_NAME': 'Sold To Customer Name',
    'CATEGORY': 'Category',
    'VOLUME': 'Volume',
    'with vat': 'Value',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel Type'
}, inplace=True)

export_sales_order_final = export_sales_order[[
    'RD Name',
    'SO Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer Number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Value',
    'FS',
    'Channel Type'
]].copy()

so_df_export = export_sales_order_final
so_df_export_final = so_df_export.sort_values(by=['SO Date', 'Sold To Customer Number'])

# SERVED INVOICE Finalization

# Step 1: Export to Excel with a blank row at the top
temp_output = 'temp_file_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    so_df_export_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 15):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 15):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save changes
full_path_so = os.path.join(export_path, ex_filename_so)

ws.sheet_view.showGridLines = False
wb.properties.author = "Joshua Ocampo"
wb.save(full_path_so)

In [11]:
# SERVED INVOICE Arrangements

ser_inv = inv_f.copy()
ser_inv_df = ser_inv.drop(['BAD RETURNS', 'GOOD RETURNS'], axis=1)

cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'ACCOUNT CODE'
})

ser_inv_df1 = ser_inv_df.merge(cml_df, on='ACCOUNT CODE', how='left')

ser_inv_f_l1 = ser_inv_df1.merge(category_df,  on='SKU CODE', how='left')

ser_inv_f_l2 = ser_inv_f_l1.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

ser_inv_f_l3 = ser_inv_f_l2.merge(week_df, on='DATE', how='inner')

ser_inv_f_l3['SKU CODE'] = ser_inv_f_l3['SKU CODE'].str.replace('_old', '', regex=False)

ser_inv_f_l4 = ser_inv_f_l3.merge(pl_m0_final_reference, on='SKU CODE', how='left')

ser_inv_f_l4['SKU PRICE REFERENCE'] = pd.to_numeric(ser_inv_f_l4['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
ser_inv_f_l4['VOLUME'] = 0.0
mask = ser_inv_f_l4['SKU PRICE REFERENCE'] != 0
ser_inv_f_l4.loc[mask, 'VOLUME'] = ser_inv_f_l4.loc[mask, 'SERVED INVOICE'] / ser_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']

ser_inv_f_l4['RD NAME'] = 'Kimberlin'


export_ser_inv = ser_inv_f_l4.copy()

export_ser_inv.rename(columns={
    'RD NAME': 'RD Name',
    'DATE': 'Invoice Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'ACCOUNT CODE': 'Sold To Customer Number',
    'CUSTOMER_NAME': 'Sold To Customer Name',
    'CATEGORY': 'Category',
    'SKU CODE': 'Product Code',
    'SKU NAME': 'Product Description',
    'VOLUME': 'Volume',
    'SERVED INVOICE': 'Value',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel Type'
}, inplace=True)

export_ser_inv_final = export_ser_inv[[
    'RD Name',
    'Invoice Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer Number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Value',
    'FS',
    'Channel Type'
]].copy()

ser_inv_df_export = export_ser_inv_final
ser_inv_df_export_final = ser_inv_df_export.sort_values(by=['Invoice Date', 'Sold To Customer Number'])

# SERVED INVOICE Finalization

# Step 1: Export to Excel with a blank row at the top
temp_output = 'temp_file_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    ser_inv_df_export_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 15):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 15):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save changes
full_path_ser_inv = os.path.join(export_path, ex_filename_ser_inv)

ws.sheet_view.showGridLines = False
wb.properties.author = "Joshua Ocampo"
wb.save(full_path_ser_inv)

In [12]:
# DMS Sellout Export

# DMS Sellout - Invoice Preparation

sellout_inv_df = df1[['Invoice Date', 'Invoice number', 'Employee ID', 'Sold To Customer Number',
                       'Sold To Customer Name', 'Item Code', 'Product/Item Description',
                        'Product UOM', 'Quantity', 'Invoice Item Type']]
sellout_inv_df_pr = sellout_inv_df[sellout_inv_df['Invoice Item Type'] != 'ITM_SALES_TAX']
sellout_inv_df_pr = sellout_inv_df_pr.drop('Invoice Item Type', axis=1)

# DMS Sellout - Customer Returns Preparation

sellout_ret_df = df2[['Customer Return Date', 'Customer Return Number', 'Sales Rep ID',
                      'Sold To Customer Number', 'Sold To Customer Name', 'Product Code',
                      'Product Description', 'UOM', 'Return/ QC Quantity', 'Customer Return Type']]
sellout_ret_df['Return/ QC Quantity'] = -abs(sellout_ret_df['Return/ QC Quantity'])  # Add this line
sellout_ret_df_pr = sellout_ret_df[sellout_ret_df['Customer Return Type'] != 'CUSTOMER_RETURN']
sellout_ret_df_pr = sellout_ret_df_pr.drop('Customer Return Type', axis=1)

# Filter Item Code/Product Code containing 'KNE'.
if dummy_code == "Y":  
    sellout_inv_df_kne = sellout_inv_df_pr[sellout_inv_df_pr['Item Code'].astype(str).str.contains('KNE', na=False)]
    sellout_ret_df_kne = sellout_ret_df_pr[sellout_ret_df_pr['Product Code'].astype(str).str.contains('KNE', na=False)]

# Include all items in the data frame.
if dummy_code == "N": 
    sellout_inv_df_kne = sellout_inv_df_pr
    sellout_ret_df_kne = sellout_ret_df_pr

#Rename columns
sellout_inv_df_kne.rename(columns={
'Invoice Date': 'Transaction Date',
'Invoice number': 'Document Number',
'Item Code': 'Product Code'
}, inplace=True)

sellout_ret_df_kne.rename(columns={
'Customer Return Date': 'Transaction Date',
'Customer Return Number': 'Document Number',
'Sales Rep ID': 'Employee ID',
'Product Description': 'Product/Item Description',
'UOM': 'Product UOM',
'Return/ QC Quantity': 'Quantity'
}, inplace=True)


# Join Invoice and Customer Returns
sellout_net_df = pd.concat([sellout_inv_df_kne, sellout_ret_df_kne], ignore_index=True)

sellout_net_df_final = sellout_net_df[['Document Number', 'Transaction Date', 'Employee ID',
                                       'Sold To Customer Number', 'Sold To Customer Name',
                                       'Product Code', 'Product/Item Description', 'Product UOM', 'Quantity']]

# Export Net DMS Sellout
ex_filename_sellout_net = "NET DMS Sellout.xlsx"
full_path_sellout_net = os.path.join(export_path, ex_filename_sellout_net)

export_sellout_net = sellout_net_df_final.copy()
export_sellout_net = export_sellout_net.sort_values(by='Transaction Date')
export_sellout_net.to_excel(full_path_sellout_net, index=False)

# Set author metadata
wb_sellout = load_workbook(full_path_sellout_net)
wb_sellout.properties.author = "Joshua Ocampo"
wb_sellout.save(full_path_sellout_net)

C:\Users\User\AppData\Local\Temp\ipykernel_15444\3899950439.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sellout_ret_df['Return/ QC Quantity'] = -abs(sellout_ret_df['Return/ QC Quantity'])  # Add this line
C:\Users\User\AppData\Local\Temp\ipykernel_15444\3899950439.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sellout_inv_df_kne.rename(columns={
C:\Users\User\AppData\Local\Temp\ipykernel_15444\3899950439.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

In [13]:
# def send_outlook_email(subject, message_body, to_list, cc_list, attachment_path):
#     import pythoncom
#     pythoncom.CoInitialize()
#     try:
#         # Create a generated dispatch (more reliable than dynamic Dispatch)
#         outlook = win32.gencache.EnsureDispatch("Outlook.Application")
#     except Exception:
#         outlook = win32.Dispatch("Outlook.Application")

#     mail = outlook.CreateItem(0)  # 0 = olMailItem
#     mail.Subject = subject
#     mail.To = ";".join(to_list)
#     mail.CC = ";".join(cc_list)

#     # Attach only if file exists
#     try:
#         if attachment_path and os.path.exists(attachment_path):
#             mail.Attachments.Add(attachment_path)
#     except Exception:
#         pass

#     # Display to load default signature, then prepend message
#     mail.Display()
#     # ensure inspector/display completed
#     signature = mail.HTMLBody or ""
#     mail.HTMLBody = f"<p>{message_body}</p><br>" + signature

#     # Leave displayed for user to review; use mail.Send() to send directly
#     # mail.Send()
#     pythoncom.CoUninitialize()
#     return mail

In [14]:
# email_date = report_date.strftime("%B %d, %Y").replace(" 0", " ")  # remove leading zero

# attachment_path = Net_Invoiced_path

# subject = f"KENEA NET INVOICED and SALES ORDERS as of {email_date}"
# message_body = f"""
# Hello ma'am/sir,<br><br>
# Here are the Net Invoiced and the Sales Order for both Nueva Ecija and Aurora, updated as of {email_date}.<br><br>
# Please see the attached files.<br><br>
# Thank you.
# """

# to_list = ["ads_sy@yahoo.com"]
# cc_list = ["raz_kenea@yahoo.com", "kimberlinaurora@gmail.com", "melvinbumanglag0109@gmail.com",
#            "niloevangelista95@gmail.com", "treblaoiprac107@gmail.com", "quennie.edquilag@urc.com.ph",
#            "joseisaac.delacruz@urc.com.ph", "raymund.gallardo@urc.com.ph", "Michelle.Lamsis@urc.com.ph",
#            "michaelcruz0308@gmail.com", "kimberlin.ccms@gmail.com", "jaycojusi020491@gmail.com", "elizabethbaltazar574@gmail.com"]


# send_outlook_email(
#     subject,
#     message_body,
#     to_list,
#     cc_list,
#     attachment_path
# )
